In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
from Datasets.CASIA2 import CASIA2
from Datasets.Utilities.Maps.Noiseprint.noiseprint import normalize_noiseprint

In [ ]:
#Download and prepare the dataset
#This will take a while since we have to process each image singularly to extract the noise features
from Geneartors.CASIA2.Casia2Generator import Casia2Generator
from Models.Customs.BaseClassifier import BaseClassifier
#from Models.Customs.ClassifierType1 import ClassifierType1
from Utilities.Plots import plot_model_data
dataset = CASIA2()
dataset.download_and_prepare()

Dl Completed...: |          | 0/0 [00:00<?, ? url/s]

Dl Size...: |          | 0/0 [00:00<?, ? MiB/s]

Extraction completed...: |          | 0/0 [00:00<?, ? file/s]

142it [00:00, 47287.91it/s]


/home/andry/anaconda3/envs/mlproject/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41487
  warnings.warn(
/home/andry/anaconda3/envs/mlproject/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  warnings.warn(


Found 4529 images of size (256, 384, 3)
Found 1898 images of size (384, 256, 3)
Found 2 images of size (428, 425, 3)
Found 1 images of size (600, 900, 3)
Found 3 images of size (347, 600, 3)
Found 18 images of size (160, 240, 3)
Found 2 images of size (382, 579, 3)
Found 38 images of size (402, 600, 3)
Found 14 images of size (316, 550, 3)
Found 74 images of size (600, 800, 3)
Found 2 images of size (587, 600, 3)
Found 42 images of size (375, 500, 3)
Found 1 images of size (332, 600, 3)
Found 2 images of size (344, 600, 3)
Found 2 images of size (552, 800, 3)
Found 3 images of size (498, 366, 3)
Found 1 images of size (599, 800, 3)
Found 6 images of size (507, 800, 3)
Found 1 images of size (400, 584, 3)
Found 28 images of size (518, 800, 3)
Found 2 images of size (439, 700, 3)
Found 1 images of size (342, 640, 3)
Found 14 images of size (480, 640, 3)
Found 7 images of size (533, 800, 3)
Found 5 images of size (312, 550, 3)
Found 17 images of size (521, 800, 3)
Found 8 images of size (

Found 1 images of size (510, 669, 3)
Found 2740 images of size (256, 384, 3)
Found 3 images of size (391, 659, 3)
Found 570 images of size (384, 256, 3)
Found 168 images of size (480, 640, 3)
Found 4 images of size (318, 600, 3)
Found 1 images of size (514, 724, 3)
Found 3 images of size (318, 528, 3)
Found 3 images of size (311, 640, 3)
Found 5 images of size (756, 504, 3)
Found 3 images of size (320, 589, 3)
Found 59 images of size (390, 561, 3)
Found 56 images of size (375, 500, 3)
Found 61 images of size (450, 600, 3)
Found 11 images of size (406, 600, 3)
Found 2 images of size (619, 480, 3)
Found 1 images of size (638, 336, 3)
Found 143 images of size (600, 800, 3)
Found 1 images of size (402, 419, 3)
Found 1 images of size (611, 399, 3)
Found 5 images of size (366, 555, 3)
Found 3 images of size (500, 334, 3)
Found 11 images of size (503, 800, 3)
Found 57 images of size (402, 600, 3)
Found 12 images of size (512, 768, 3)
Found 36 images of size (521, 800, 3)
Found 1 images of siz

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: |          | 0/? [00:00<?, ? examples/s]

In [ ]:
train_split = dataset.as_dataset(split="train")

n_cols = 3

nsamples = 5

samples = train_split.take(nsamples)

col_titles = ['Image','Noiseprint','SRM'] 

nrows = nsamples
ncols = 3

fig, axes = plt.subplots(nrows,ncols,figsize=(12,18))  # create the figure with subplots
[ax.set_axis_off() for ax in axes.ravel()]  # remove the axis

for ax, col in zip(axes[0], col_titles): # set up the title for each column
    ax.set_title(col,fontdict={'fontsize':18,'color':'b'})

i = 0
for sample in samples:
    axes[i,0].imshow(sample["image"])
    axes[i,1].imshow(normalize_noiseprint(sample["noiseprint"].numpy()))
    axes[i,2].imshow(sample["SRM"])   
    i = i + 1 
      

In [ ]:
#Define parameters essentials for the training of the models

#Define input parameters
input_shape_rgb = (256, 384, 3)
input_shape_rbf = (256, 384, 3)
input_shape_noiseprint = (256, 384, 1)

#We just have to distinguish between tampered and pristine images
#and a single class is enough for that
output_classes = 1

#Define the loss the models will use
loss_function = "binary_crossentropy"

#Define the number of epochs each model has to be trained for
epochs = 1

#define the size of each training batch
batch_size = 32

In [ ]:
#Define additional parameters not essentials for the training

#Set the path to the Log folder in which the logs, the checkpoints and other usefull
#data will be used
logs_folder = Path("../Logs")

#Set verbose = True if you want an extensive printing of logs during the training
# and testing of the models
verbose = True

In [ ]:
# Create 2 generator of datas that has that provide samples with the following structure:
#   X -> [RGB image]
#   Y -> class of the image
# The first generator will produce training data, the second will produce validation data

generator_training_rgb = Casia2Generator(dataset.as_dataset(split="train"),["rgb"],batch_size)
generator_validation_rgb = Casia2Generator(dataset.as_dataset(split="validation"),["rgb"],batch_size)
#Train a Resnet Classifier using the RGB data
model_rgb = BaseClassifier(input_shape_rgb,output_classes,"RGB model",logs_folder,verbose)
history_rgb , rgb_model_path, rgb_checkpoint_path = model_rgb.train_model(generator_training_rgb,generator_validation_rgb,epochs,loss_function,save_model=True)

In [ ]:
plot_model_data(history_rgb,("accuracy","val_accuracy"),("training  accuracy","validation accuracy"),"Rgb model training")

In [ ]:
# Create 2 generator of datas that has that provide samples with the following structure:
#   X -> [Noiseprint image]
#   Y -> class of the image
# The first generator will produce training data, the second will produce validation data

generator_training = Casia2Generator(dataset.as_dataset(split="train"),["noiseprint"],batch_size)
generator_validation = Casia2Generator(dataset.as_dataset(split="validation"),["noiseprint"],batch_size)

#Train a Resnet Classifier using the Noiseprint data
model_noiseprint = BaseClassifier(input_shape_noiseprint,output_classes,"Noiseprint model",logs_folder,verbose)
history_noiseprint , noiseprint_model_path,noiseprint_checkpoint_path =  model_noiseprint.train_model(generator_training,generator_validation,epochs,loss_function,save_model=True)

In [ ]:
plot_model_data(history_noiseprint,("accuracy","val_accuracy"),("training  accuracy","validation accuracy"),"Rgb model training")